In [1]:
import pandas as pd 
pd.options.display.float_format = '{:.4f}'.format
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm, t
from scipy.interpolate import CubicSpline

import cufflinks as cf
cf.set_config_file(offline=True)
cf.set_config_file(theme = 'pearl')

In [2]:
# install a package for producing sobol sequences
# pip install sobol_seq 

In [3]:
import sobol_seq

### Uploading data

In [4]:
### read all neccessary data

Corr_t = pd.read_csv('Kendall corr.csv',header=[0], index_col=[0])
Corr_gaus = pd.read_csv('Pearson corr.csv',header=[0], index_col=[0])
curve = pd.read_csv('Disc_curve.csv',header=[0], index_col=[0])
Survival_data = pd.read_csv('Survival_data.csv',header=[0], index_col=[0])

In [5]:
names = ['UAE','QATAR','SAUDI','OMAN','BAHRAIN']

#### Interpolation of hazard and survival rates up to 10Y

In [6]:
# interpolate new years for survival rates
h_tenors = Survival_data.Maturity.unique() # tenors of hazard rates

# code for interpolation of hazard and survival rates
def get_interp_rate(y, tenor, x=h_tenors):
    spline = CubicSpline(x,y)
    return spline(tenor)

In [7]:
def Survival_prepare():
    # Survival rate Term structure
    s_data = [np.hstack(Survival_data.Survival[Survival_data.Country==i]) for i in names]
    Survival = pd.DataFrame(s_data, columns = [str(i)+'Y' for i in Survival_data.Maturity.unique()] , index = names )

    new_tenor = ['6.0Y','8.0Y','9.0Y']
    add_df = pd.DataFrame(columns = new_tenor, index = names)
    for i in range(len(Survival.values)):
        add_df.iloc[i,0] = get_interp_rate(Survival.values[i],6).item()
        add_df.iloc[i,1] = get_interp_rate(Survival.values[i],8).item()
        add_df.iloc[i,2] = get_interp_rate(Survival.values[i],9).item()

    Survival = pd.concat([Survival,add_df], axis = 1)
    Survival = Survival[['0.0Y', '1.0Y', '2.0Y', '3.0Y', '4.0Y', '5.0Y','6.0Y','7.0Y','8.0Y','9.0Y','10.0Y']]
    return Survival

In [8]:
def Hazard_prepare():
    # Hazard rate Term structure
    h_data = [np.hstack(Survival_data.Hazard[Survival_data.Country==i]) for i in names]
    Hazard = pd.DataFrame(h_data, columns = [str(i)+'Y' for i in Survival_data.Maturity.unique()] , index = names)

    new_tenor = ['6.0Y','8.0Y','9.0Y']
    add_df = pd.DataFrame(columns = new_tenor, index = names)
    for i in range(len(Hazard.values)):
        add_df.iloc[i,0] = get_interp_rate(Hazard.values[i],6).item()
        add_df.iloc[i,1] = get_interp_rate(Hazard.values[i],8).item()
        add_df.iloc[i,2] = get_interp_rate(Hazard.values[i],9).item()

    Hazard = pd.concat([Hazard,add_df], axis = 1)
    Hazard = Hazard[['0.0Y', '1.0Y', '2.0Y', '3.0Y', '4.0Y', '5.0Y','6.0Y','7.0Y','8.0Y','9.0Y','10.0Y']]
    return Hazard

In [9]:
Survival = Survival_prepare()
Survival

,0.0Y,1.0Y,2.0Y,3.0Y,4.0Y,5.0Y,6.0Y,7.0Y,8.0Y,9.0Y,10.0Y
UAE,1.0000,0.9977,0.9931,0.9865,0.9776,0.9647,0.9488,0.9313,0.9134,0.8963,0.8812
QATAR,1.0000,0.9975,0.9926,0.9850,0.9751,0.9620,0.9448,0.9249,0.9044,0.8853,0.8699
SAUDI,1.0000,0.9964,0.9906,0.9818,0.9712,0.9566,0.9405,0.9237,0.9061,0.8874,0.8675
OMAN,1.0000,0.9922,0.9805,0.9629,0.9357,0.9035,0.8680,0.8309,0.7942,0.7599,0.7300
BAHRAIN,1.0000,0.9862,0.9587,0.9252,0.8863,0.8390,0.7933,0.7514,0.7126,0.6761,0.6411


In [10]:
Hazard = Hazard_prepare()
Hazard

,0.0Y,1.0Y,2.0Y,3.0Y,4.0Y,5.0Y,6.0Y,7.0Y,8.0Y,9.0Y,10.0Y
UAE,0.0000,0.0024,0.0046,0.0066,0.0092,0.0132,0.0162,0.0176,0.0181,0.0182,0.0184
QATAR,0.0000,0.0027,0.0049,0.0077,0.0101,0.0135,0.0170,0.0197,0.0214,0.0217,0.0204
SAUDI,0.0000,0.0045,0.0059,0.0089,0.0108,0.0152,0.0175,0.0175,0.0169,0.0174,0.0210
OMAN,0.0000,0.0096,0.0118,0.0181,0.0287,0.0350,0.0390,0.0419,0.0435,0.0439,0.0432
BAHRAIN,0.0000,0.0176,0.0283,0.0356,0.0430,0.0548,0.0587,0.0551,0.0494,0.0469,0.0529


### Set of functions for Basket CDS pricing

In [11]:
# Code for log-linear interpolation (taken from CQF Python lab 17)

def get_discount_factor(tenor,maturity=curve.maturity, discountfactor = curve.Disc_factor):
    
    max_time_index = len(maturity) - 1
    
    if tenor == 0: Df = 1.
    if tenor > 0 and tenor < maturity[0]: Df = discountfactor[0]
    if tenor >= maturity[max_time_index]: Df = discountfactor[max_time_index]
        
    for i in range(0, max_time_index):
         if tenor >= maturity[i] and tenor < maturity[i+1]:
            term1 = ((tenor-maturity[i])/(maturity[i+1] - maturity[i]))*np.log(discountfactor[i+1])
            term2 = ((maturity[i+1]-tenor)/(maturity[i+1] - maturity[i]))*np.log(discountfactor[i])
            lnDf = term1 + term2
            Df = np.exp(lnDf)
            
    return Df

In [12]:
# We use spectral matrix decomposition as it doesn't require correlation matrix to be positive definite

def impose_correlation_spectral(random_sample, covar_matrix):
    # Perform spectral decomposition of the covar_matrix
    eigenvalues, eigenvectors = np.linalg.eigh(covar_matrix)
    
    # Obtain the square root of eigenvalues
    sqrt_eigen = np.diag(np.sqrt(eigenvalues))
    
    # Transform the random sample using the spectral decomposition to impose correlation
    correlated_sample = np.dot(np.dot(eigenvectors, sqrt_eigen), np.dot(eigenvectors.T, random_sample))
    
    # check
    # print(np.dot(np.dot(eigenvectors, sqrt_eigen),np.dot(eigenvectors, sqrt_eigen).T))
    
    return correlated_sample

In [13]:
# Simulate gaussian copula
# Can choose whether to use sobol sequence or default RNG

def simulate_Gaussian(num_iter, corr, sobol = False):
    num_items = corr.shape[0]
    
    if sobol == True:
        Z = sobol_seq.i4_sobol_generate_std_normal(num_items, num_iter).T    
    else:
        Z = np.random.randn(num_items, num_iter)
            
    X = impose_correlation_spectral(Z, corr)
    U = norm.cdf(X)
    return U

In [14]:
# Simulate gaussian copula
# Can choose whether to use sobol sequence or default RNG
# df is at 3 as per maximum likelihood estimation done earlier

def simulate_T(num_iter, corr, sobol = False,  df = 3):
    num_items = corr.shape[0]
    
    if sobol == True:
        Z = sobol_seq.i4_sobol_generate_std_normal(num_items, num_iter).T
    else:
        Z = np.random.randn(num_items, num_iter)

    Z2 = np.random.randn(df, num_iter)    
    s = np.sum(Z2**2, axis = 0)
    Y = Z/((s/df)**0.5)
    X = impose_correlation_spectral(Y, corr)
    U = t.cdf(X,df)
    return U

In [15]:
# Chart representing Sobol sequences 

# import seaborn as sns
# sob = pd.DataFrame(sobol_seq.i4_sobol_generate_std_normal(2,250), columns = ['a','b'])
# sns.pairplot(sob)

In [16]:
# Compute actual default time
# U: pseudo random uniform sample
def time_of_default(hazard,survival,U):
    
    
    T = 1000 # arbitrary lagre number
    year_of_default = []
    dt = []
    
    # find the year of default
    for i in range(len(hazard)):
        flag=0
        for j in range(len(hazard.values[i])):
            if np.sum(hazard.values[i][0:j]) >= np.abs(np.log(1-U[i])): # for each in name j
                year_of_default.append(j-1)
                flag=1
                break
                
        if flag==0:
            year_of_default.append(T) # no default occures
    
    
    for i in range(len(year_of_default)):
        if year_of_default[i] == T: # no default occurs
            dt.append(0) # no need to append anything
        else:
            # year fraction to be added     
            dt.append((-1/hazard.values[i][year_of_default[i]])*np.log((1-U[i])/survival.values[i][year_of_default[i]-1])) 

    time_of_default = np.array(year_of_default)+np.array(dt)-1 #compensate that haz. and surv. rates start from year 0
    year_of_default = [i - 1 for i in year_of_default] #compensate that haz. and surv. rates start from year 0
    
    return time_of_default, year_of_default

In [17]:
# Payment and default leg computation for k-th to default CDS
# k: k-th to default
# with no leg removal when pricing 2nd and more defaults

def payments(time_of_default, year_of_default, k, RR, maturity, IS, L):
    
    paydates = [i for i in range(1,maturity+1)] # assuming annual paymants for CDS
    items = len(time_of_default)
    
    # Disregarding defaults happend just after CDS contract signed. Changing default year and time if time goes beyound CDS term
    for i in range(len(time_of_default)):
        if time_of_default[i] < 0.25:
            time_of_default[i] = 0.25 # floor
        if time_of_default[i] > maturity:
            time_of_default[i] = 999
            year_of_default[i] = 999
    

    def_order = np.argsort(time_of_default) # yields index which are sorted by time of default
    n_defaults = len(year_of_default)-year_of_default.count(999) # compute actual num of defaults in this iteration
    premium_leg = 0
    default_leg = 0
    
    # we assume no leg removal when pricing 2nd and more defaults       

    if n_defaults == 0 or n_defaults < k: #check  whether there is enough defaults happend
        for k in range(items):
            premium_leg += get_discount_factor(paydates[k])
            default_leg = 0
    else:
        tau = time_of_default[def_order[k-1]]
        premium_leg = tau*get_discount_factor(tau)
        if IS == True:
            default_leg = (1-RR)/items*get_discount_factor(tau)*L[def_order[k-1]]
            
        else:
            default_leg = (1-RR)/items*get_discount_factor(tau)
       
    return premium_leg, default_leg

In [18]:
# Compute Fair Spreads of k-th to default basket CDS by sampling from Gaussian-Copula
# k: k-th to default
# corr: correlation

def fair_spread_MC(k, copula, corr, RR, maturity, hazard, survival, num_iter, sobol, IS):
    Pleg_sum = 0
    DLeg_sum = 0
    
    if copula == 'g':
        # sampling from Gaussian copula
        U = simulate_Gaussian(num_iter, corr, sobol).T #one time similation and then slice
        
    elif copula == 't':
        U = simulate_T(num_iter, corr, sobol, df = 3).T
           
    for i in range(0, num_iter):
        if IS == True and copula == 'g': # IS is only done for Gaussian copula
            V,L = IS_sampling(k,corr, hazard, survival, U[i])
            time_default, year_default = time_of_default(hazard,survival,V) # get default times
            premium_leg, default_leg = payments(time_default, year_default, k, RR, maturity, IS, L) # get legs
            Pleg_sum += premium_leg
            DLeg_sum += default_leg
        else:
            L = 1
            IS = False
            time_default, year_default = time_of_default(hazard,survival,U[i]) # get default times
            premium_leg, default_leg = payments(time_default, year_default, k, RR, maturity, IS, L) # get legs
            Pleg_sum += premium_leg
            DLeg_sum += default_leg
       

    Pleg_average = Pleg_sum/num_iter
    Dleg_average = DLeg_sum/num_iter

    spread = Dleg_average/Pleg_average
    
    
    return spread*10000 #Convert to bps

In [19]:
# calculates condition probabilities for Importance Sampling as per ..
def cond_probabilities(Survival, cor, U):
    
    Z = norm.ppf(U)
    PD5Y = 1-Survival['5.0Y']
    PD_con = np.zeros(len(PD5Y))
    A = np.linalg.cholesky(cor)
    if np.all(np.linalg.eigvals(cor) <= 0): 
        print('Corr matrix is not positive definite')
    for i in range(len(PD5Y)):
        if i==0:
            PD_con[i] = PD5Y[i]
        else:

            PD_con[i] = norm.cdf((norm.ppf(PD5Y[i])-np.dot(A[i,0:i],Z[0:i]))/A[i,i])
    return PD_con

# norm.cdf(norm.ppf(PD5Y[0])/A[0,0])
# norm.cdf((norm.ppf(PD5Y[1])-sum(A[1,0]*Z[0]))/A[1,1])
# norm.cdf((norm.ppf(PD5Y[2])-sum(A[2,0]*Z[0]+A[2,1]*Z[1]))/A[2,2])
# norm.cdf((norm.ppf(PD5Y[3])-sum(A[3,0]*Z[0]+A[3,1]*Z[1]+A[3,2]*Z[2]))/A[3,3])
# norm.cdf((norm.ppf(PD5Y[4])-sum(A[4,0]*Z[0]+A[4,1]*Z[1]+A[4,2]*Z[2]+A[4,3]*Z[3]))/A[4,4])

In [20]:
#IS_sampling(1, Corr_gaus, Hazard, Survival, np.random.rand(5))[0]

In [21]:
#time_of_default(Hazard,Survival,IS_sampling(1, Corr_gaus, Hazard, Survival, np.random.rand(5))[0])

In [22]:
def IS_sampling(n, cor, Hazard, Survival, U):

    indic = [1 if x<=5 else 0 for x in time_of_default(Hazard, Survival, U)[0]] 
    N = len(indic)
    PD_IS = np.zeros(N)
    L = np.zeros(N)

    V = np.zeros(N)
    count = 0
    

    
    PD_cond = cond_probabilities(Survival, cor, U)
    
    for i in range(N):
        if i==0:
            PD_IS[i] = n/N
            if PD_IS[i] == 1:
                count+=1
        else:
            if sum(indic[0:i],count)>=n:
                PD_IS[i] = PD_cond[i]
            else:
                PD_IS[i] = (n-sum(indic[0:i],count))/(N-i) # can be adjsuted as per Chen and Glasserman
                if PD_IS[i] == 1:
                    count+=1
    #print(indic)            
    #print(PD_cond)
    #print(PD_IS)
    
    U_ = np.random.rand(N)
    

    for i in range(N):
        if U_[i] <= PD_IS[i]:
            L[i] = PD_cond[i]/PD_IS[i]
            V[i] = L[i]*U_[i]
        else:
            L[i] = (1-PD_cond[i])/(1-PD_IS[i])
            V[i] = PD_cond[i]+L[i]*(U_[i]-PD_IS[i])

    
    return V,L

### Price Basket CDS: results

In [23]:
method = ['Gaussian', 'T']
kth = ['1st-to-def.','2nd-to-def.','3rd-to-def.','4th-to-def.','5th-to-def.']



spreads = pd.DataFrame(columns = method, index = kth)

for i in range(1,6):
    spreads.iloc[i-1,0]=fair_spread_MC(i, 'g', Corr_gaus, 0.4, 5, Hazard, Survival, 10000, sobol = False, IS = False)
    spreads.iloc[i-1,1]=fair_spread_MC(i, 't', Corr_t, 0.4, 5, Hazard, Survival, 10000, sobol = False, IS = False)


In [24]:
spreads

,Gaussian,T
1st-to-def.,57.5775,55.1500
2nd-to-def.,23.2424,22.0149
3rd-to-def.,8.5514,9.1346
4th-to-def.,3.6206,5.7190
5th-to-def.,1.1685,2.9620


### Price for Sobol seq. approach

In [25]:

method = ['Gaussian', 'T']
kth = ['1st-to-def.','2nd-to-def.','3rd-to-def.','4th-to-def.','5th-to-def.']



spreads = pd.DataFrame(columns = method, index = kth)

for i in range(1,6):
    spreads.iloc[i-1,0]=fair_spread_MC(i, 'g', Corr_gaus, 0.4, 5, Hazard, Survival, 10000, sobol = True, IS = False)
    spreads.iloc[i-1,1]=fair_spread_MC(i, 't', Corr_t, 0.4, 5, Hazard, Survival, 10000, sobol = True, IS = False)


In [26]:
spreads

,Gaussian,T
1st-to-def.,59.4133,54.8822
2nd-to-def.,22.6429,21.0139
3rd-to-def.,8.2667,8.7306
4th-to-def.,3.8008,5.5392
5th-to-def.,1.4603,2.9109


### Convergence for Sobol seq. and Default RNG

In [48]:
iter_s = [100, 200, 500, 750, 1000, 1500, 2000, 3000, 4000, 6000]#, 7000, 10000.]
method = ['Gaussian RNG', 'Gaussian Sobol']
kth = ['1st-to-def.','2nd-to-def.','3rd-to-def.','4th-to-def.','5th-to-def.']

allnames = []
for i in range(len(method)):
    for j in range(len(kth)):
        allnames.append(method[i]+' '+kth[j])

spreads = pd.DataFrame(columns = allnames, index = iter_s)

for i in range(1,len(kth)+1):
    print(i)
    for j in range(len(iter_s)):
        spreads.iloc[j,i-1]=fair_spread_MC(i, 'g', Corr_gaus, 0.4, 5, Hazard, Survival, iter_s[j], sobol = False, IS = False)
        spreads.iloc[j,i-1+5]=fair_spread_MC(i, 'g', Corr_gaus, 0.4, 5, Hazard, Survival, iter_s[j], sobol = True, IS = False)
        


1
2
3
4
5


In [49]:
spreads.loc[:,'Gaussian Sobol 1st-to-def.':].iplot(title='Convergence of pricing for Sobol (Gaussian copula)', xTitle='iterations', yTitle='CDS price bps')

In [50]:
spreads.loc[:,:'Gaussian RNG 5th-to-def.'].iplot(title='Convergence of pricing for default RNG (Gaussian copula)', xTitle='iterations', yTitle='CDS price bps')

In [51]:
spreads

,Gaussian RNG 1st-to-def.,Gaussian RNG 2nd-to-def.,Gaussian RNG 3rd-to-def.,Gaussian RNG 4th-to-def.,Gaussian RNG 5th-to-def.,Gaussian Sobol 1st-to-def.,Gaussian Sobol 2nd-to-def.,Gaussian Sobol 3rd-to-def.,Gaussian Sobol 4th-to-def.,Gaussian Sobol 5th-to-def.
100,57.4055,19.4591,19.6544,2.2370,2.4851,45.6983,11.6195,0.0000,0.0000,0.0000
200,61.0485,27.9722,2.2567,7.0490,2.4224,55.2932,18.9043,2.2645,1.1387,0.0000
500,59.8784,26.0733,7.6703,3.7829,1.8605,61.0350,20.0481,5.4942,0.9159,0.4516
750,63.2700,23.3159,6.6104,2.2036,0.9222,61.5289,22.1881,5.8600,2.1456,0.9076
1000,61.7662,25.0283,7.4050,3.6172,0.6881,61.9860,23.1520,6.2629,2.7509,1.1295
1500,59.1490,26.4181,8.8752,2.9585,1.2542,60.0879,23.3554,7.3455,3.3959,1.3739
2000,58.4067,23.1117,9.4227,3.4001,1.6307,59.5128,22.9201,7.5092,3.1204,1.1455
3000,56.3052,21.0362,8.3219,3.9963,1.3142,59.5245,23.7225,8.3300,3.5500,1.3744
4000,58.8299,25.5412,7.1663,3.6696,1.0046,59.1815,23.4598,8.6811,3.7144,1.3764
6000,59.1427,22.4323,8.5760,3.7484,1.6305,59.1035,23.2518,8.4171,3.7781,1.3446


### Recovery rate effect analysis

In [31]:
RR_s = np.linspace(0.1, 0.9, 5)
method = ['Gaussian', 'T']
kth = ['1st-to-def.','2nd-to-def.','3rd-to-def.','4th-to-def.','5th-to-def.']

allnames = []
for i in range(len(method)):
    for j in range(len(kth)):
        allnames.append(method[i]+' '+kth[j])

spreads = pd.DataFrame(columns = allnames, index = RR_s)

for i in range(1,6):
    for j in range(len(RR_s)):
        spreads.iloc[j,i-1]=fair_spread_MC(i, 'g', Corr_gaus, RR_s[j], 5, Hazard, Survival, 4000, sobol = False, IS = False)
        spreads.iloc[j,i-1+5]=fair_spread_MC(i, 't', Corr_t, RR_s[j], 5, Hazard, Survival, 4000, sobol = False, IS = False)


In [32]:
spreads.loc[:,'T 1st-to-def.':].iplot(title='Recovery rate effect', xTitle='Recovery rate', yTitle='CDS price bps')

In [33]:
spreads.loc[:,:'Gaussian 5th-to-def.'].iplot(title='Recovery rate effect', xTitle='Recovery rate', yTitle='CDS price bps')

In [34]:
spreads

,Gaussian 1st-to-def.,Gaussian 2nd-to-def.,Gaussian 3rd-to-def.,Gaussian 4th-to-def.,Gaussian 5th-to-def.,T 1st-to-def.,T 2nd-to-def.,T 3rd-to-def.,T 4th-to-def.,T 5th-to-def.
0.1000,90.9615,35.1772,13.4800,4.6865,1.6675,83.6927,36.4448,14.2981,7.7240,4.1627
0.3000,64.6306,28.5679,9.1150,4.2709,1.5635,63.5071,25.8637,10.0319,6.4129,3.0643
0.5000,49.4286,19.2023,8.0851,3.6299,1.0204,41.8435,17.9145,8.5831,4.5797,2.5673
0.7000,28.0863,10.0197,3.6480,2.1130,0.4641,26.9864,10.8607,4.6028,2.3108,1.4595
0.9000,10.1525,3.7808,1.4168,0.6576,0.2558,9.2123,3.5331,1.7496,0.8464,0.3782


### Correlation effect analysis

In [35]:
## corr matrix effect analysis

times = np.linspace(0.0,1.5,6)
method = ['Gaussian', 'T']
kth = ['1st-to-def.','2nd-to-def.','3rd-to-def.','4th-to-def.','5th-to-def.']

allnames = []
for i in range(len(method)):
    for j in range(len(kth)):
        allnames.append(method[i]+' '+kth[j])

spreads = pd.DataFrame(columns = allnames, index = times )

for i in range(1,6):
    for j in range(len(times)):
        spreads.iloc[j,i-1]=fair_spread_MC(i, 'g', np.where(Corr_gaus==1,Corr_gaus,np.minimum(Corr_gaus*times[j],0.95)), 0.4, 5, Hazard, Survival, 4000, sobol = False, IS = False)   
        spreads.iloc[j,i-1+5]=fair_spread_MC(i, 't', np.where(Corr_t==1,Corr_t,np.minimum(Corr_t*times[j],0.95)), 0.4, 5, Hazard, Survival, 4000, sobol = False, IS = False)


In [36]:
spreads.loc[:,'T 1st-to-def.':].iplot(title='Correlation effect', xTitle='corr. matrix multiplier', yTitle='CDS price bps')

In [37]:
spreads.loc[:,:'Gaussian 5th-to-def.'].iplot(title='Correlation effect', xTitle='corr. matrix multiplier', yTitle='CDS price bps')

In [38]:
spreads

,Gaussian 1st-to-def.,Gaussian 2nd-to-def.,Gaussian 3rd-to-def.,Gaussian 4th-to-def.,Gaussian 5th-to-def.,T 1st-to-def.,T 2nd-to-def.,T 3rd-to-def.,T 4th-to-def.,T 5th-to-def.
0.0000,93.5380,10.2623,0.4627,0.0000,0.0000,80.8874,14.8281,3.2325,0.6582,0.0000
0.3000,87.3282,14.4056,2.5026,0.4591,0.0559,72.8283,19.9245,5.4052,1.0659,0.0589
0.6000,73.3393,18.5632,5.1125,1.2914,0.1162,65.8673,19.1319,6.0639,2.9691,0.6500
0.9000,60.9762,22.5108,7.2807,3.2229,1.0485,58.9903,23.2434,8.4035,4.2206,2.0939
1.2000,51.2964,24.9477,9.0401,4.7521,3.2863,48.8805,23.9361,11.2236,7.6404,3.7483
1.5000,46.0066,27.7455,11.6903,6.9850,3.8239,44.8043,26.6504,10.8742,7.7390,5.8290


### Credit spread effect (on all constituents)

We do parallel shifts (of +25bps, +50bps, +100bps, +200bps, +4000bps) for Hazard rates, then recalculate Survavial rates and then do monte carlo to get Basket CDS spreads

In [39]:
Surv = Survival.copy()
Haz = Hazard.copy()

Haz_s = [25,25,50,100,200]
method = ['Gaussian', 'T']
kth = ['1st-to-def.','2nd-to-def.','3rd-to-def.','4th-to-def.','5th-to-def.']

allnames = []
for i in range(len(method)):
    for j in range(len(kth)):
        allnames.append(method[i]+' '+kth[j])

spreads = pd.DataFrame(columns = allnames, index = [25,50,100,200,400])


for i in range(1,6):
    j = 0
    for k in Haz_s:
        Haz.loc[:,'1.0Y':] = Haz.loc[:,'1.0Y':]+k/10000
        for m in range(Haz.loc[:,'1.0Y':].shape[1]):
            Surv.iloc[:,m+1] = np.exp(-np.sum(np.array(Haz.iloc[:,:m+2]).astype(float), axis = 1))

        spreads.iloc[j,i-1]=fair_spread_MC(i, 'g', Corr_gaus, 0.4, 5, Haz, Surv, 4000, sobol = False, IS = False)
        spreads.iloc[j,i-1+5]=fair_spread_MC(i, 't', Corr_t, 0.4, 5, Haz, Surv, 4000, sobol = False, IS = False)
        j+=1   

In [40]:
spreads

,Gaussian 1st-to-def.,Gaussian 2nd-to-def.,Gaussian 3rd-to-def.,Gaussian 4th-to-def.,Gaussian 5th-to-def.,T 1st-to-def.,T 2nd-to-def.,T 3rd-to-def.,T 4th-to-def.,T 5th-to-def.
25,65.8389,95.8257,109.6198,105.6809,90.6397,64.3237,96.9149,105.5707,104.5829,90.6163
50,71.9510,104.4439,113.1399,105.5804,86.0813,68.7868,100.9412,115.1502,108.9958,89.6249
100,91.0006,117.0448,118.4129,114.7723,91.2426,84.2057,109.8329,118.3713,109.6883,91.5603
200,114.1053,130.2608,132.1122,119.8704,97.7225,107.8727,131.0509,132.4112,127.1136,99.6847
400,174.0613,174.0691,162.8836,146.4333,115.4916,167.0067,169.7788,157.5467,141.3714,117.4736


In [41]:
spreads.loc[:,'T 1st-to-def.':].iplot(title='Credit spread effect (shift for all)', xTitle='Shift up, bps', yTitle='CDS price bps')

In [42]:
spreads.loc[:,:'Gaussian 5th-to-def.'].iplot(title='Credit spread effect (shift for all)', xTitle='Shift up, bps', yTitle='CDS price bps')

In [43]:
spreads

,Gaussian 1st-to-def.,Gaussian 2nd-to-def.,Gaussian 3rd-to-def.,Gaussian 4th-to-def.,Gaussian 5th-to-def.,T 1st-to-def.,T 2nd-to-def.,T 3rd-to-def.,T 4th-to-def.,T 5th-to-def.
25,65.8389,95.8257,109.6198,105.6809,90.6397,64.3237,96.9149,105.5707,104.5829,90.6163
50,71.9510,104.4439,113.1399,105.5804,86.0813,68.7868,100.9412,115.1502,108.9958,89.6249
100,91.0006,117.0448,118.4129,114.7723,91.2426,84.2057,109.8329,118.3713,109.6883,91.5603
200,114.1053,130.2608,132.1122,119.8704,97.7225,107.8727,131.0509,132.4112,127.1136,99.6847
400,174.0613,174.0691,162.8836,146.4333,115.4916,167.0067,169.7788,157.5467,141.3714,117.4736


### Credit spread effect (on a single name)

same as above but shift only UAE

In [44]:
Surv = Survival.copy()
Haz = Hazard.copy()

Haz_s = [25,25,50,100,200]
method = ['Gaussian', 'T']
kth = ['1st-to-def.','2nd-to-def.','3rd-to-def.','4th-to-def.','5th-to-def.']

allnames = []
for i in range(len(method)):
    for j in range(len(kth)):
        allnames.append(method[i]+' '+kth[j])

spreads = pd.DataFrame(columns = allnames, index = [25,50,100,200,400])


for i in range(1,6):
    j = 0
    Haz = Hazard.copy()
    for k in Haz_s:
        Haz.loc['UAE','1.0Y':] = Haz.loc['UAE','1.0Y':]+k/10000
        for m in range(Haz.loc[:,'1.0Y':].shape[1]):
            Surv.iloc[0,m+1] = np.exp(-np.sum(Haz.iloc[0,:m+2]))

        spreads.iloc[j,i-1]=fair_spread_MC(i, 'g', Corr_gaus, 0.4, 5, Haz, Surv, 4000, sobol = False, IS = False)
        spreads.iloc[j,i-1+5]=fair_spread_MC(i, 't', Corr_t, 0.4, 5, Haz, Surv, 4000, sobol = False, IS = False)
        j+=1 

In [45]:
spreads.loc[:,'T 1st-to-def.':].iplot(title='Credit spread effect (shift for single)', xTitle='Shift up, bps', yTitle='CDS price bps')

In [46]:
spreads.loc[:,:'Gaussian 5th-to-def.'].iplot(title='Credit spread effect (shift for single)', xTitle='Shift up, bps', yTitle='CDS price bps')

In [47]:
spreads

,Gaussian 1st-to-def.,Gaussian 2nd-to-def.,Gaussian 3rd-to-def.,Gaussian 4th-to-def.,Gaussian 5th-to-def.,T 1st-to-def.,T 2nd-to-def.,T 3rd-to-def.,T 4th-to-def.,T 5th-to-def.
25,60.9257,23.9874,9.7627,2.3493,1.2748,54.9566,23.7566,10.4399,5.3857,3.7347
50,63.2420,24.6355,10.4693,4.3465,1.8339,57.0725,25.2030,11.7240,6.7295,3.1006
100,64.2100,25.8090,11.2885,4.4404,1.4129,64.1962,25.8180,13.2554,6.7608,2.9892
200,66.6039,28.7049,13.0635,5.1428,2.3998,66.2663,31.1248,14.6723,6.9833,3.5609
400,89.5168,36.9096,14.8620,5.1959,1.9819,84.9494,36.9496,16.4757,6.7554,3.4068
